## Adquisición y preprocesamiento de la información

<table>
    <tr>
      <td>Grado en Ingeniería de Datos e Inteligencia Artificial - Facultad de Informática - UCM
      </td>
      <td>
      <img src="https://biblioteca.ucm.es/data/cont/media/www/pag-88746//escudo.jpg"  width=50/>
      </td>
     </tr>
</table>




### Un poquito de Web Scraping

cuando los datos no son fáciles de obtener de una página web debemos analizar directemente el código HTML de la práctica. Para esto podemos usar, entre otras, dos bibliotecas:

- BeautifulSoup: el estándar para manejar directamente código HTML. Permite navegar por los elementos de la página de forma sencilla
- Selenium: cuando la página requiere interactividad, y se quiere automatizar la pulsación de botones, selección de listas desplegables, etc.


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://www.rffm.es/competiciones/clasificaciones?season=18&type=2&grouping=1&competition=15293079&group=16544990&round="


# error, no hay tablas, por eso usamos BeautifulSoup.
#dfs = pd.read_html(page.text)

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice

soup

<!DOCTYPE html>

<html lang="es">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Clasificaciones | Real Federación de Fútbol de Madrid</title>
<meta content="Clasificaciones" name="description"/>
<meta content="https://www.rffm.es/competiciones/clasificaciones" property="og:url">
<meta content="article" property="og:type">
<meta content="Clasificaciones | Real Federación de Fútbol de Madrid" property="og:title">
<meta content="Clasificaciones" property="og:description"/>
<meta content="" property="og:image"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@RFFM_oficial" name="twitter:site"/>
<meta content="Clasificaciones" name="twitter:title"/>
<meta content="Clasificaciones" name="twitter:description"/>
<meta content="@RFFM_oficial" name="twitter:creator"/>
<meta content="" name="twitter:image"/>
<meta content="https://www.rffm.es" name="twitter:domain"/>
<!-- style -->
<link href="/css/style.min

El paso anterior ha analizado el texto y extraído sus componentes. Ahora podemos escribir, por ejemplo:

In [32]:
print(soup.head.title)

<title>Clasificaciones | Real Federación de Fútbol de Madrid</title>


In [33]:
print(soup.head.title.text)

Clasificaciones | Real Federación de Fútbol de Madrid


Esta forma de "navegar" la página de arriba a abajo puede resultar muy tediosa y además solo permite llegar a algunos elementos, es la llamada "dot navegation" que usaremos cuando ya estemos cerca de la información que queremos

In [37]:
soup.body.div

<div class="search-popup" style="display:none;">
<!-- CROSS ICON -->
<svg class="cross-icon big close-button search-popup-close">
<use xlink:href="#svg-cross-big"></use>
</svg>
<!-- /CROSS ICON -->
<form action="https://www.rffm.es/noticias" class="search-popup-form" method="GET">
<input class="input-line" id="search" name="q" placeholder="¿Qué estás buscando?" type="text"/>
</form>
<p class="search-popup-text">¡Escribe lo que estás buscando y presiona Enter para comenzar tu búsqueda!</p>
</div>

Para buscar con más libertad tendremos otras funciones como 'find' y 'find_all' que encuentran, respectivamente, la primera aparición o todas las apariciones de un tag:

In [43]:
cabecera = soup.find('h1')
cabecera

<h1 class="section-title big">
                                    TEMPORADA 2022-2023
                                                    | PREFERENTE PREBENJAMIN
                                                    | GRUPO 5
                                                    | JORNADA 11
                                                    | 10-02-2023
                            </h1>

In [45]:
cabeceras = soup.find_all('h2')
len(cabeceras)

34

In [48]:
cabeceras[2]

<h2 class="section-title small">Estructura organizativa</h2>

Ahora vamos a buscar la información que nos interesa. Tras inspeccionar la página en google Chrome (por ejemplo), vemos que parece que cada fila viene en un elemento de tipo `div` con atributo  class="table-row". La función `select`devuelve todos los elementos que cumplen esto (nótese el . antes del nombre)

In [54]:
row = soup.select('.table-row')
len(row)

24

In [67]:
for i in range(len(row)):
    print(i,row[i].text.strip().replace("\n", ""))

0 1                                                                                                                            C.D. ESTUDIANTES LAS TABLAS 'A'                                                                                                                                28                                                                                                                                                11                                                                                                                                                9                                                                                                                                                1                                                                                                                                                1                                                                                                                                   

Dentro de cada fila obtendremos los datos uno a uno. El nombre del equipo está en un elemento team-name, como solo hay uno usamos `select_one`

In [68]:
team_name = row[0].select_one('.team-name')
team_name

<p class="team-name">
<a href="https://www.rffm.es/competiciones/equipos?team=3909479&amp;season=18&amp;type=2&amp;grouping=1&amp;competition=15293079&amp;group=16544990&amp;round=&amp;club=">
                                                        C.D. ESTUDIANTES LAS TABLAS 'A'
                                                    </a>
</p>

In [23]:
a = team_name.find("a")
a

<a href="https://www.rffm.es/competiciones/equipos?team=3909479&amp;season=18&amp;type=2&amp;grouping=1&amp;competition=15293079&amp;group=16544990&amp;round=&amp;club=">
                                                        C.D. ESTUDIANTES LAS TABLAS 'A'
                                                    </a>

In [24]:
a.text

"\n                                                        C.D. ESTUDIANTES LAS TABLAS 'A'\n                                                    "

In [25]:
a.text.strip()

"C.D. ESTUDIANTES LAS TABLAS 'A'"

In [52]:
team_name.a.text.strip()

"C.D. ESTUDIANTES LAS TABLAS 'A'"

Los elementos de cada fila:

In [53]:
datos = row[0].select('.table-row-item')
print(len(datos))

9


In [28]:
datos[0].text.strip()

"1\n                                                                    \n\n\n\n\n\n\n\n\n\n\n                                                        C.D. ESTUDIANTES LAS TABLAS 'A'"

In [29]:
datos[1].text.strip()

'28'

Ya podemos trabajar

In [89]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://www.rffm.es/competiciones/clasificaciones?season=18&type=2&grouping=1&competition=15293079&group=16544990&round="


page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
rows =  soup.select('.table-row') # las filas



cols = {"Equipos":[],"Puntos":[],"Jugados":[],"Ganados":[],"Empates":[],"Perdidos":[], "GF":[],"GC":[],"PuntosSancion":[]}
keys = list(cols.keys())

for f in range(len(rows)//2): # lo de /2 es porque parece que se repiten
    row = rows[f]
    team_name = row.select_one('.team-name')
    cols["Equipos"].append(team_name.a.text.strip())
    datos = row.select('.table-row-item') # datos de la fila
    
    # completamos columna a columna
    for c in range(1,len(datos)):
        nombre = keys[c] # la clave (Puntos, etc)
        valor = int(datos[c].text.strip())
        cols[nombre].append(valor)

In [90]:
import pandas as pd
df = pd.DataFrame(cols)
df

Equipos  Puntos  Jugados  Ganados  Empates  \
0      C.D. ESTUDIANTES LAS TABLAS 'A'      28       11        9        1   
1                 UNION ZONA NORTE 'B'      25       11        8        1   
2                   A.D. ESPERANZA 'A'      25       11        8        1   
3                 REAL MADRID C.F. 'B'      24       11        7        3   
4       GREDOS SAN DIEGO-MORATALAZ 'A'      20       11        6        2   
5                    C.D. CANILLAS 'B'      19       11        6        1   
6             A.D. OÑA SANCHINARRO 'A'      18       11        6        0   
7                 CLUB SAN AGUSTIN 'A'      10       11        3        1   
8                    A.D.A.E. SIMANCAS       9       11        3        0   
9   ALAMEDA DE OSUNA ESCUELAS DE F 'A'       9       11        3        0   
10      C.D. OLIMPICO DE HORTALEZA 'A'       3       11        0        3   
11               A.D. UNION ADARVE 'B'       1       11        0        1   

    Perdidos  GF  GC  PuntosSancion  
0          1  58  17              0  
1          2  49  17              0  
2          2  40  17              0  
3          1  66  15              0  
4          3  35  27              0  
5          4  52  39              0  
6          5  30  33              0  
7          7  38  44              0  
8          8  22  43              0  
9          8  22  43              0  
10         8  15  52              0  
11        10   4  84              0